In [1]:
def list_blobs(bucket_name, folder_name):
    """List all files in given COS directory."""    
    blob_names = []
    gcs_client = storage.Client()
    bucket = gcs_client.bucket(bucket_name)
    blobs = list(bucket.list_blobs(prefix=folder_name))
    for blob in blobs:
        blob_names.append(blob.name)
    return blob_names
    
def list_blobs_pd(bucket_name, folder_name):
    """List all files in given COS directory."""       
    gcs_client = storage.Client()
    bucket = gcs_client.bucket(bucket_name)
    blobs = list(bucket.list_blobs(prefix=folder_name))

    blob_name = []
    blob_size = []
    blob_time = []
    
    for blob in blobs:
        blob_name.append(blob.name)
        blob_size.append(blob.size)
        blob_time.append(blob.time_created)

    blobs_df = pd.DataFrame(list(zip(blob_name, blob_size, blob_time)), columns=['filePath', 'size', 'timeStamp'])    
    return blobs_df

def download_blob(bucket_name, source_blob_name, destination_file_name):
    """Downloads a blob from COS bucket."""
    gcs_client = storage.Client()
    bucket = gcs_client.bucket(bucket_name)
    blob = bucket.blob(source_blob_name)
    blob.download_to_filename(destination_file_name)

def upload_blob(bucket_name, source_file_name, destination_blob_name):
    """Uploads a file to the bucket."""    
    gcs_client = storage.Client()
    bucket = gcs_client.bucket(bucket_name)
    blob = bucket.blob(destination_blob_name)
    blob.upload_from_filename(source_file_name)

In [2]:
import sqlite3
from google.cloud import storage
import pandas as pd

In [3]:
DB_NAME = "test.db"
BUCKET_NAME = "data_tql"
FOLDER_NAME = "WikiSQL/data/"

In [4]:
blobs = list_blobs_pd(BUCKET_NAME, FOLDER_NAME)

In [5]:
list_blobs(BUCKET_NAME, FOLDER_NAME)

['WikiSQL/data/dev.db',
 'WikiSQL/data/dev.jsonl',
 'WikiSQL/data/dev.tables.jsonl',
 'WikiSQL/data/test.db',
 'WikiSQL/data/test.jsonl',
 'WikiSQL/data/test.tables.jsonl',
 'WikiSQL/data/train.db',
 'WikiSQL/data/train.jsonl',
 'WikiSQL/data/train.tables.jsonl']

In [6]:
DB_NAME = "test.db"
BUCKET_NAME = "data_tql"
FOLDER_NAME = "WikiSQL/data"

storage_client = storage.Client()

bucket = storage_client.bucket(BUCKET_NAME)
blob = bucket.blob(f"{FOLDER_NAME}/{DB_NAME}")

# Download the SQLite file to a local file
local_file_name = "test.db"
blob.download_to_filename(local_file_name)

# Connect to the SQLite file using sqlite3
conn = sqlite3.connect(local_file_name)

In [7]:
cursor = conn.cursor()
cursor.execute("SELECT * FROM sqlite_master WHERE type='table';")
schemas = cursor.fetchall()

In [15]:
schemas[0]

('table',
 'table_1_10015132_16',
 'table_1_10015132_16',
 2,
 'CREATE TABLE table_1_10015132_16 (col0 text, col1 text, col2 text, col3 text, col4 text, col5 text)')

In [16]:
BUCKET_NAME = "data_tql"
FOLDER_NAME = "wikisqliteDB"

for schema in schemas[0]:
    # Get the schema name and create a new SQLite file for it
    schema_name = schema[1]
    query = schema[4]
    new_db_name = f"{schema_name}.sqlite"
    new_conn = sqlite3.connect(new_db_name)
    cursor = new_conn.cursor()

    # Export the schema to the new SQLite file
    cursor.execute(query)
    new_conn.commit()
    
    # Upload the SQLite file to GCP
    storage_client = storage.Client()
    bucket = storage_client.bucket(BUCKET_NAME)
    blob = bucket.blob(f"{FOLDER_NAME}/{new_db_name}")
    blob.upload_from_filename(new_db_name)

    # Close the connection to the new SQLite file
    new_conn.close()

# Close the connection to the .db file
conn.close()


OperationalError: near "e": syntax error

In [19]:
schema = schemas[0]

In [20]:
schema

('table',
 'table_1_10015132_16',
 'table_1_10015132_16',
 2,
 'CREATE TABLE table_1_10015132_16 (col0 text, col1 text, col2 text, col3 text, col4 text, col5 text)')

In [21]:
schema_name = schema[1]
query = schema[4]
new_db_name = f"{schema_name}.sqlite"
new_conn = sqlite3.connect(new_db_name)
cursor = new_conn.cursor()

    # Export the schema to the new SQLite file
cursor.execute(query)
new_conn.commit()
    
    # Upload the SQLite file to GCP
storage_client = storage.Client()
bucket = storage_client.bucket(BUCKET_NAME)
blob = bucket.blob(f"{FOLDER_NAME}/{new_db_name}")
blob.upload_from_filename(new_db_name)

    # Close the connection to the new SQLite file
new_conn.close()
conn.close()
